In [ ]:
import os
import requests
import json

url = "http://localhost:11434/api/generate"

In [3]:
def llama3(prompt:str, sample:int = 0.3):
    
    data = {
        "model": "llama3.1:8b-instruct-q8_0",
        "prompt": prompt,
        "system": "You are a highly capable causal inference expert. Your primary task is to identify and analyze causal relationships between variables, focusing on direct cause-and-effect connections between those variables. Act as a sceptical causal reasoning agent, systematically solving causality problems using logical analysis and evidence-based reasoning",
        "stream": False,
        "format": "json",
        "options": {
            "top_p": sample # (0-1) 0 is no sampling // api default is 0.9 // testing seems good with 0.3
        }
    }

    response = requests.post(url, json=data)
    return(response.json()["response"])

### Helper-Functions

In [ ]:
def direction_finder(connection):
    #solves directionality question for connection
    
    var1 = connection[0]
    var2 = connection[1]

    json_template = '{"cause": "var", "effect: "var"}'
    prompt_start = f"Given two provided variables that are causally related, identify which variable is the cause and which is the effect. You must use the exact variables as provided without introducing or modifying any variables. Clearly label one variable as 'cause' and the other as 'effect' based strictly on causal reasoning. "
    prompt_end = f"Ensure the output strictly adheres to the following JSON format: {json_template}. The two provided variables are: '{var1}' and '{var2}'." #If the two variables are too loosly connected to form any kind auf cause and effect relationship, simply replace both variable names with 'NONE' in the output json."
    prompt = prompt_start + prompt_end

    response = llama3(prompt, sample=0)

    #print(prompt)

    with open("log_dump/direction.txt", "a") as f:
        f.write("\n--------------------------------------------------------------\n")
        tmp = f"var1: {var1} var2: {var2} = \n"
        f.write(tmp)
        f.write(response)

    rep = json.loads(response)

    if rep["cause"] == var1 and rep["effect"] == var2:
        out = "->"
    elif rep["effect"] == var1 and rep["cause"] == var2:
        out = "<-"
    else:
        print("wrong typing", rep)
        out = ""


    return out

In [6]:
def safe(filename: str, input):
    #simple write for later use

    with open(filename + ".txt", 'w') as file:
        file.write(json.dumps(input))

In [7]:
def load(path:str, old:bool = False):
    #simple load with backwards compatability

    if old == False:
        with open(path) as file:
            got = json.loads(file.read())
    else:
        log = []
        with open(path) as file:
            for line in file:
                tmp = line.replace("'", "\"")
                tmp2 = json.loads(tmp)
                log.append(tmp2)
            
        got = {
            "example": ["no", "no"],
            "final_graph": log
        }
    
    return got

### MAIN

### Control Area

In [ ]:
# example Backlog

tubing = load("manual_safes/datasets/tubing_clean.json")
tubing_examples = []

for i in tubing:
    #print(tubing[i])
    var1 = tubing[i]["var1"].replace(","," -")      #replaces , for - because llm mistakes it as sepereate elements otherwise
    var2 = tubing[i]["var2"].replace(","," -")
    tubing_examples.append({"start": var1, "end":var2, "dir": tubing[i]["direction"]})
#pair 52 not workable: x and y are both 4-dimensional variables for day
#pair 53,54 vars are list of things

print("amount tubing example", len(tubing_examples))

amount own example 16
amount tubing example 91


In [13]:
for i in tubing_examples:
    print(i)

{'start': 'Altitude', 'end': 'Temperature', 'dir': '->'}
{'start': 'Altitude', 'end': 'Precipitation', 'dir': '->'}
{'start': 'Longitude', 'end': 'Temperature', 'dir': '->'}
{'start': 'Altitude', 'end': 'Sunshine hours', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Length', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Shell weight', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Diameter', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Height', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Whole weight', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Shucked weight', 'dir': '->'}
{'start': 'Abalone Age', 'end': 'Viscera weight', 'dir': '->'}
{'start': 'Age', 'end': 'Wage per hour', 'dir': '->'}
{'start': 'Displacement', 'end': 'Fuel consumption', 'dir': '->'}
{'start': 'Horse power', 'end': 'Fuel consumption', 'dir': '->'}
{'start': 'Weight', 'end': 'Fuel consumption', 'dir': '->'}
{'start': 'Horsepower', 'end': 'Acceleration', 'dir': '->'}
{'start': 'Age', 'end': 'Dividends from stocks

In [ ]:
#parameters
run_automation = 1
examples = tubing_examples

chosen_example =  2     #to use without automation
run_from = 1          #to use with automation
run_to = 108            #applied like ls[run_from-1:run_to] 
nth_element = 1        #runs only every Nth element from the above selected

In [ ]:
#execute direct tubing
open("log_dump/direction.txt", "w").close()

if run_automation:
    example_sublist = examples[run_from-1:run_to] 
    example_sublist = example_sublist[0::nth_element]
    num_runs = len(example_sublist)
    run_id = 0

    num_correct = 0

    for example in example_sublist:
        start_var = example["start"]
        end_var = example["end"]
        dir = example["dir"]


        direct = direction_finder([start_var, end_var])               #returns "->"

        print(start_var, end_var)
        if dir == direct:
            print("correct")
            num_correct += 1
        else:
            print("false", dir, direct)

        run_id += 1
        

    print("------------End of Execution------------")
    print(f"got correct: {num_correct}/{num_runs}")

Altitude Temperature
correct
Altitude Precipitation
false -> <-
Longitude Temperature
false -> <-
Altitude Sunshine hours
correct
Abalone Age Length
correct
Abalone Age Shell weight
correct
Abalone Age Diameter
correct
Abalone Age Height
correct
Abalone Age Whole weight
correct
Abalone Age Shucked weight
correct
Abalone Age Viscera weight
correct
Age Wage per hour
correct
Displacement Fuel consumption
false -> <-
Horse power Fuel consumption
false -> <-
Weight Fuel consumption
false -> <-
Horsepower Acceleration
false -> <-
Age Dividends from stocks
correct
Age Concentration GAG
correct
duration of eruption time to the next eruption
correct
Latitude Temperature
correct
Longitude Precipitation
false -> <-
Age Height
false -> <-
Age Weight
correct
Age Heart rate
correct
Cement content in concrete Compressive strength
correct
Blast furnace slag content in concrete Compressive strength
correct
Fly ash content in concrete Compressive strength
correct
Water content in concrete Compressive st